# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> #### Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Módulo

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [14]:
#include <stdio.h>

int main() {
    float S = 200;
    int N = 5;
    float J = 1;
    float parcela = S;
    
    for (int i = 1; i <= N; i++) {
        printf("O valor da parcela %d eh %.2f\n", i, parcela);
        parcela = parcela*(1 + (J/100));
    }   
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [3]:
#include <stdio.h>

float self_S;
float self_J;
int self_N;
float parcela;
int atual;

float novo_emprestimo(float S, float J, int N){
    self_S = S;
    self_J = J;
    self_N = N;
    atual = 1;
    parcela = S;
    
}

float proximaParcela(){
    float retorno = parcela;
    atual++;
    if(atual <= self_N)
        parcela = parcela*(1 + self_J/100);
    else
        parcela = 0;
    
    return retorno;
}

int main(){
    novo_emprestimo(200, 1, 5);
    int i = 1;
    float parcela = proximaParcela();
    while(parcela > 0){
        printf("Parcela %d: %.2f\n", i, parcela);
        parcela = proximaParcela();
        i++;
    }
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [2]:
// Usar Ponteiros

#include <stdio.h>

float novo_emprestimo(float S, float J, int N, int *atual, float *parcela){
    *atual = 1;
    *parcela = S;
}

void proximaParcela(float S, float J, int N, int *atual, float *parcela){
    (*atual)++;
    if(*atual <= N)
        *parcela = *parcela*(1 + J/100);
    else
        *parcela = 0;
}

int main(){
    int atual;
    float parcela;

    novo_emprestimo(200, 1, 5, &atual, &parcela);

    int i = 1;
    while(parcela > 0){
        printf("Parcela %d: %.2f\n", i, parcela);
        proximaParcela(200, 1, 5, &atual, &parcela);
        i++;
    }
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [11]:
// Usar Struct

#include <stdio.h>
#include <stdlib.h>

typedef struct emprestimo {
    float S;
    float J;
    int N;
    int atual;
    float parcela;
} Emprestimo;

Emprestimo *novo_emprestimo(Emprestimo *emprestimo){
    emprestimo->atual = 1;
    emprestimo->parcela = emprestimo->S;
    return emprestimo;
}

float proximaParcela(Emprestimo *emprestimo) {
    emprestimo->atual++;
    float retorno = emprestimo->parcela;
    if(emprestimo->atual <= emprestimo->N)
        emprestimo->parcela = emprestimo->parcela*(1 + emprestimo->J/100);
    else
        emprestimo->parcela = 0;
    return retorno;
}

int main(){
    Emprestimo *emprestimo = malloc(sizeof(Emprestimo));
    emprestimo->S = 200;
    emprestimo->J = 1;
    emprestimo->N = 5;
    emprestimo = novo_emprestimo(emprestimo);
    
    int i = 1;
    float parcela = proximaParcela(emprestimo);
    while(parcela > 0){
        printf("Parcela %d: %.2f\n", i, parcela);
        parcela = proximaParcela(emprestimo);
        i++;
    }
    free(emprestimo);
}

Parcela 1: 200.00
Parcela 2: 202.00
Parcela 3: 204.02
Parcela 4: 206.06
Parcela 5: 208.12


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [12]:
#include <stdio.h>
#include <stdlib.h>

typedef struct emprestimo {
    float S;
    float J;
    int N;
    int atual;
    float parcela;
} Emprestimo;

Emprestimo *novo_emprestimo(float S, float J, int N) {
    Emprestimo *emprestimo = malloc(sizeof(Emprestimo));
    emprestimo->S = S;
    emprestimo->J = J;
    emprestimo->N = N;
    emprestimo->atual = 1;
    emprestimo->parcela = S;
    return emprestimo;
}

float proximaParcela(Emprestimo *emprestimo) {
    emprestimo->atual++;
    float retorno = emprestimo->parcela;
    if(emprestimo->atual <= emprestimo->N)
        emprestimo->parcela = emprestimo->parcela*(1 + emprestimo->J/100);
    else
        emprestimo->parcela = 0;
    
    return retorno;
}


int main(){
    Emprestimo *emprestimo1, *emprestimo2;
    
    emprestimo1 = novo_emprestimo(200, 1, 5);
    emprestimo2 = novo_emprestimo(500, 2, 7);
    
    float parcela1 = proximaParcela(emprestimo1);
    float parcela2 = proximaParcela(emprestimo2);
    int i = 1;
    
    while(parcela1 || parcela2 > 0){
        if(parcela1 > 0){
            printf("Empréstimo 1: parcela %d eh %.2f\n", i, parcela1);
            parcela1 = proximaParcela(emprestimo1);
        }
        if(parcela2 > 0){
            printf("Empréstimo 2: parcela %d eh %.2f\n", i, parcela2);
            parcela2 = proximaParcela(emprestimo2);
        }
        i++;
    }
    free(emprestimo1);
    free(emprestimo2);
    
}

Empréstimo 1: parcela 1 eh 200.00
Empréstimo 2: parcela 1 eh 500.00
Empréstimo 1: parcela 2 eh 202.00
Empréstimo 2: parcela 2 eh 510.00
Empréstimo 1: parcela 3 eh 204.02
Empréstimo 2: parcela 3 eh 520.20
Empréstimo 1: parcela 4 eh 206.06
Empréstimo 2: parcela 4 eh 530.60
Empréstimo 1: parcela 5 eh 208.12
Empréstimo 2: parcela 5 eh 541.22
Empréstimo 2: parcela 6 eh 552.04
Empréstimo 2: parcela 7 eh 563.08
